In [1]:
import re
import time
import random
import requests
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities

C:\Users\boo\anaconda3\envs\py36\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.17) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import datetime as dt

# 저장할 리스트 초기화
URLs = {'Trump': [], 'Harris': []}

# 시작 날짜와 종료 날짜
startdate = dt.date(year=2024, month=8, day=31)
untildate = dt.date(year=2024, month=9, day=1)

# 종료일이 2024년 9월 30일이 될 때까지 반복
while untildate != dt.date(year=2024, month=10, day=1):
    # 트럼프 트윗 검색 URL
    url_Trump = 'https://x.com/search?q=(from%3ArealDonaldTrump)%20lang%3Aen%20until%3A' + str(untildate) + '%20since%3A' + str(startdate) + '&src=typed_query'
    
    # 카말라 해리스 트윗 검색 URL
    url_Harris = 'https://x.com/search?q=(from%3AVP)%20lang%3Aen%20until%3A' + str(untildate) + '%20since%3A' + str(startdate) + '&src=typed_query'
    
    # 다음 날짜로 이동
    startdate = untildate
    untildate += dt.timedelta(days=1)
    
    # URL 리스트에 추가
    URLs['Trump'].append(url_Trump)
    URLs['Harris'].append(url_Harris)

# 출력: 생성된 URL을 확인
for key, value in URLs.items():
    print(f"{key}: {len(value)} URLs generated")


Trump: 30 URLs generated
Harris: 30 URLs generated


In [3]:
url_df=pd.DataFrame(URLs)
url_df.head()

,Trump,Harris
0,https://x.com/search?q=(from%3ArealDonaldTrump...,https://x.com/search?q=(from%3AVP)%20lang%3Aen...
1,https://x.com/search?q=(from%3ArealDonaldTrump...,https://x.com/search?q=(from%3AVP)%20lang%3Aen...
2,https://x.com/search?q=(from%3ArealDonaldTrump...,https://x.com/search?q=(from%3AVP)%20lang%3Aen...
3,https://x.com/search?q=(from%3ArealDonaldTrump...,https://x.com/search?q=(from%3AVP)%20lang%3Aen...
4,https://x.com/search?q=(from%3ArealDonaldTrump...,https://x.com/search?q=(from%3AVP)%20lang%3Aen...


In [4]:
url_df['Trump'][0]

'https://x.com/search?q=(from%3ArealDonaldTrump)%20lang%3Aen%20until%3A2024-09-01%20since%3A2024-08-31&src=typed_query'

In [5]:
url_df['Harris'][19]

'https://x.com/search?q=(from%3AVP)%20lang%3Aen%20until%3A2024-09-20%20since%3A2024-09-19&src=typed_query'

In [35]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from progressbar import ProgressBar
from time import sleep
import random

# Chromedriver 경로 설정 및 브라우저 초기화
browser_path = "/Users/boo/USA_President_Election/chromedriver" 
driver = webdriver.Chrome(executable_path=browser_path)   

In [37]:
def sleep_for(opt1, opt2):
    time_for = random.uniform(opt1, opt2)
    time_for_int = int(round(time_for))
    sleep(abs(time_for_int - time_for))
    for i in range(time_for_int, 0, -1):
        sleep(1)

master_temp_df = pd.DataFrame()
master_df = pd.DataFrame()

for urls in url_df['Trump'][:]:
    pbar = ProgressBar()
    urls=[urls]
    post_element_xpath = '//div/div/article/div/div' # the element we are obtaining from the webpage
    for url in pbar(urls):                          # loop through your list of urls
        driver.get(url)
        sleep_for(3, 7)                             # sleep a while

        lastHeight = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.find_element_by_xpath('//body').send_keys(Keys.END)
            sleep_for(2, 6)
            newHeight = driver.execute_script("return document.body.scrollHeight")
            if newHeight != lastHeight:
                pass
            else:
                post_list = driver.find_elements_by_xpath(post_element_xpath)      # get a list of tweets
                post_text = [x.text for x in post_list]                           # get the text only from each element
                temp_df = pd.DataFrame(post_text, columns={'all_text'})           # create temporary dataset of each tweet
                master_temp_df = master_temp_df.append(temp_df)                   # append the temporary dataset to the dataset we will save
                break
            
            lastHeight = newHeight      # update Height
            master_df=master_df.append(master_temp_df)
            
master_df.to_csv("trump0901_0930.csv", index=False)

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

In [39]:
master_temp_df

,all_text
0,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
1,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
2,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
3,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
4,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
5,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
0,Donald J. Trump\n@realDonaldTrump\n·\nSep 1\nD...
1,Donald J. Trump\n@realDonaldTrump\n·\nSep 1\nM...
2,Donald J. Trump\n@realDonaldTrump\n·\nSep 2\nI...
3,Donald J. Trump\n@realDonaldTrump\n·\nSep 1\nC...


In [40]:
master_df=master_df.append(master_temp_df)

In [41]:
master_df

,all_text
0,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
1,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
2,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
3,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
4,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
5,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
0,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
1,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
2,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...
3,Donald J. Trump\n@realDonaldTrump\n·\nAug 31\n...


In [42]:
master_df.to_csv("trump0901_0930.csv", index=False)